In [97]:
import numpy as np
import pandas as pd
import re

file = open('data/wiki sample data.txt','r')

lines = file.readlines()
#print (lines)

df = pd.DataFrame(columns=['timestamp','user','minorEdit','pageLength','editDiff','comment','tags'])
i=0
for line in lines:
        #split via brackets
        token = re.split('\(|\)',line)
        #print(token)
        timestamp = token[2].split('?')[0]
        timestamp = pd.to_datetime(timestamp)
        #split by ?
        user = token[2].split('?')[1:]
        user = ''.join(user)
        user = user[1:-1]
        m = token[4]
        m= m.split('.')[0]
        m= True if (m=='? m ') else False
        byte = token[5].split(' ')[0]
        byte = int(byte.replace(',',''))
        change = token[7]
        change = int(change.replace(',',''))
        comment = re.split('\. \. ',line)[-1]
        tagComment = comment.split('(undo)')
        #print (tagComment)
        comment = tagComment[0][1:-2]
        #undo is not at the end
        if (len(tagComment)>1):
            tag = tagComment[1][2:-2]
            tag = tag.split(':')
            if (len(tag)>1):
                tag = tag[1][1:]
                tag = tag.split(',')
        else:
            tag=''
        if (tag[0]==''):
            tag =[]
        #tag = tag.split(',')
        print (tag)
        
        df.loc[i]=[timestamp,user,m,byte,change,comment,tag]
        i+=1

print (df.shape)

[]
[]
[]
['Mobile edit', ' Mobile web edit']
['Mobile edit', ' Mobile web edit']
['Mobile edit', ' Mobile web edit']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['references removed']
[]
[]
['Visual edit']
[]
['Mobile edit', ' Mobile web edit']
[]
['Visual edit']
[]
[]
[]
[]
[]
[]
[]
[]
['Mobile edit', ' Mobile web edit', ' references removed']
['Mobile edit', ' Mobile web edit']
['Mobile edit', ' Mobile web edit']
[]
['Mobile edit', ' Mobile web edit']
[]
['canned edit summary', ' Mobile app edit', ' Mobile edit']
[]
['Mobile edit', ' Mobile web edit']
['Visual edit']
['Visual edit']
['Mobile edit', ' Mobile web edit']
['Visual edit']
['Visual edit']
['Mobile edit', ' Mobile web edit']
['Mobile edit', ' Mobile web edit']
['Mobile edit', ' Mobile web edit']
['Mobile edit', ' Mobile web edit']
['Mobile edit', ' Mobile web edit']
['section blanking', ' Mobile edit', ' Mobile web edit']
['Mobile edit', ' Mobile web edit']
['Mobile edit', ' Mobile web edit']
[]
[]
[]
[]
[]
[]
['Visual edit']
['Visual edit'

IndexError: string index out of range